# SimPy Example for IE 306.02
SimPy is a powerful discrete-event simulation library in Python that can be used to model and simulate various systems, including queuing systems. 

This example shows how a simple M/M/1 system (1 server with an infinite capacity queue, exponential interarrival and service times) can be modeled with a process-interaction view using the SimPy library. 

In this example customers place calls to the call center of a company at random times. There is only a single operator in this call center, and picks up the first call waiting when she is available. 

We start by importing the necessary packages:

In [1]:
import simpy
import random

Define a set of globals that define the characteristics of the model instance to be simulated. This includes the seed (RANDOM_SEED) for the random number generators, total number of customers that will depart during the run of the simulation, and key parameters for the interarrival (i.e. mean arrival rate) and service time (i.e. service rate) distribution.

In [3]:
RANDOM_SEED = 978
NUMBER_OF_CUSTOMERS = 300
INTERARRIVAL_RATE = 0.1 
SERVICE_RATE = 1/5.725
random.seed(RANDOM_SEED)

Define the necessary lists for bookkeeping:

In [4]:
customers = []#List of customers
service_times = [] #Duration of the conversation between the customer and the operator (Service time)
queue_w_times = [] #Time spent by a customer while it waits for the operator (Queue waiting time Wq)

* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator resource. The customer is put on hold until the resource activates it back. 

* When the resource is available, the customer is activated and it then initiates the ask_question process. The duration of a question-answer session is determined randomly according to the service time distribution.

In [5]:
class Customer(object):
    def __init__(self, name, env, operator):
        self.env = env
        self.name = name
        self.operator = operator
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))
 
        with self.operator.request() as req:
            yield req
            print('%s is assigned to an operator at %g' % (self.name, self.env.now))
            queue_w_times.append(self.env.now - self.arrival_t)
            yield self.env.process(self.ask_question())
            print('%s is done at %g' % (self.name, self.env.now))
            
    def ask_question(self):
        duration = random.expovariate(SERVICE_RATE)
        yield self.env.timeout(duration)
        service_times.append(duration)
        
        

Create customers that will join the queue during the run of the simulation:

In [6]:
def customer_generator(env, operator):
    """Generate new customers that call the line."""
    for i in range(NUMBER_OF_CUSTOMERS):
        yield env.timeout(random.uniform(4.17,5.37))
        customer = Customer('Cust %s' %(i+1), env, operator)
        customers.append(customer)  

Finally, create the main environment, resource and process, then run the simulation:

In [7]:
env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator))
env.run()

Cust 1 initiated a call at 5.35554
Cust 1 is assigned to an operator at 5.35554
Cust 1 is done at 5.91429
Cust 2 initiated a call at 9.71153
Cust 2 is assigned to an operator at 9.71153
Cust 2 is done at 14.0739
Cust 3 initiated a call at 14.8442
Cust 3 is assigned to an operator at 14.8442
Cust 4 initiated a call at 19.537
Cust 3 is done at 24.2905
Cust 4 is assigned to an operator at 24.2905
Cust 5 initiated a call at 24.6998
Cust 6 initiated a call at 29.4363
Cust 4 is done at 29.6693
Cust 5 is assigned to an operator at 29.6693
Cust 7 initiated a call at 33.6915
Cust 8 initiated a call at 38.0866
Cust 5 is done at 41.0423
Cust 6 is assigned to an operator at 41.0423
Cust 6 is done at 42.7945
Cust 7 is assigned to an operator at 42.7945
Cust 9 initiated a call at 43.3427
Cust 7 is done at 45.3821
Cust 8 is assigned to an operator at 45.3821
Cust 10 initiated a call at 48.4967
Cust 8 is done at 52.8524
Cust 9 is assigned to an operator at 52.8524
Cust 11 initiated a call at 53.8355
C

In [8]:
print(queue_w_times, "\n")

[0.0, 0.0, 0.0, 4.753454425610897, 4.96947854115497, 11.605991479912692, 9.102926052075041, 7.295484968866951, 9.509720092043722, 12.37599962418269, 9.850094352435711, 6.145855468145996, 1.1627486440384018, 3.8205543487738254, 1.23931423877724, 0.0, 0.0, 0.32979389428911077, 3.4664774978177064, 9.674882055641476, 10.357549433060896, 8.875728935153063, 6.462569809123238, 4.086937032617087, 8.703317438733222, 5.047662599464161, 7.3870097612962695, 24.230994073000574, 23.856544573779075, 21.1810037839428, 30.952492496681117, 29.102715136200516, 25.526946066768062, 36.635195560635594, 40.09273018528441, 36.69083975921839, 35.164853087103324, 32.70090642253666, 42.29325853526311, 46.3282436553348, 43.70612548005931, 42.06247879070071, 49.4263989730255, 47.82428140400589, 49.61514917812528, 47.675692352161974, 68.54968902032957, 79.74900029562116, 79.41462483346226, 76.05272466571182, 77.01296907159733, 77.90043141411067, 80.34024139100316, 76.395551394665, 74.1531986012514, 72.9848263890687

In [9]:
print(service_times, "\n")

[0.5587423851830706, 4.362393928600156, 9.446255906635434, 5.378778679320115, 11.373053294672614, 1.752124688327702, 2.5876784236718855, 7.470239910918281, 8.02031787799962, 2.8129390486366974, 1.4844586311766903, 0.215539713623899, 7.453855335666334, 2.068003857192061, 1.1610299949451621, 4.569438984394743, 4.525095522296573, 7.427992640946125, 11.371600836440821, 5.843727965313247, 2.8018359556682384, 2.4522666830302495, 2.685278056306107, 9.04425406027848, 1.615672460566113, 6.844194228558323, 21.3657543153762, 3.8237757007903213, 1.8279438726311235, 14.30033562164434, 2.5855930086785777, 1.248373034236667, 15.364218865949947, 8.048731032358432, 1.3508327799336708, 3.6974342587895506, 2.1332156315786324, 14.384557276275872, 8.396117872032002, 2.4195569514939033, 2.7151499372565415, 12.11680820335335, 3.3706157819235587, 6.432085286772567, 2.6400533186291884, 25.07313161764333, 16.397049664888847, 3.9931280922335572, 1.9652455944608787, 6.218339321102647, 5.430392765190173, 7.6629702